# LLM skills extraction - Saudi Dataset
The goal of this notebook is to use gpt4 to extract values for soft skills and technicall skills using the current job descriptions in the Saudi Indeed dataset. This will be done in order to standardize the method of data collection of Saudi Indeed branch with the USA branch of the Job Trends project.

In [1]:
# Import relevant packages
import pandas as pd
import numpy as np
import requests
import re
import json

# Haystack imports
from typing import List
from haystack.dataclasses import Document
from haystack import Pipeline
from haystack import component
from haystack.components.builders import PromptBuilder
from haystack.components.generators.openai import OpenAIGenerator
from haystack.components.fetchers import LinkContentFetcher
from haystack.components.converters import HTMLToDocument

In [2]:
# Supress some warnings

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Load env files

from helper import load_env
load_env()

In [9]:
# Load data
data = pd.read_csv("full_llm_labeled_data_march_2025.csv") 

In [10]:
# Check label counts
data['label'].value_counts()

label
none                         722
cloud engineer               424
business intelligence        273
ai-related                   226
data engineer                146
data governance              117
data analyst                 108
data scientist                70
machine learning engineer     11
data architect                 1
Name: count, dtype: int64

In [11]:
data['label'].isna().sum()

4224

In [12]:
# Change labels outside the scope to "Other"

data['label'].unique()

data['label'] = np.where(data['label'].isin(['data governance', 
                                             'business intelligence', 
                                             'ai-related', 
                                             'data engineer', 
                                             'machine learning engineer', 
                                             'data scientist', 
                                             'cloud engineer', 
                                             'data analyst']), data['label'], 'Other')

In [13]:
# Check if it worked
data['label'].value_counts()

label
Other                        4947
cloud engineer                424
business intelligence         273
ai-related                    226
data engineer                 146
data governance               117
data analyst                  108
data scientist                 70
machine learning engineer      11
Name: count, dtype: int64

In [14]:
# Build a haystack component that can be used to fetch job descriptions from the Dataframe

@component
class DescriptionFetcher:

    '''
    Fetch information from a row in the dataset and convert it to json format, which can be subsequently embedded and used by a llm model.
    '''

    @component.output_types(job_info=str)
    def run(self, df, row_number: int): # The component requires a DataFrame and a row number (integer position based)
        return {"job_info": {df.iloc[row_number][["description"]].to_json()}}

In [17]:
# Set the template for query to be equivalent to the one used by USA branch

query_template = """
    You are an expert in extracting both explicit and implicit skills from job descriptions, with a particular focus on accuracy and detail.
    Your task is to extract **all** relevant skills and job-related details, ensuring comprehensive coverage, especially for data-related professions.
    
    1. Identify Soft Skills:
        - Extract non-technical skills, such as communication, teamwork, problem-solving, and leadership.
        - Include both explicit and implied soft skills (e.g., "collaborative environment" suggests teamwork).
        - Normalize skill names (e.g., "Excellent communication skills" should be "communication").
        - List all identified soft skills, separated by commas.
    
    2. Identify Technical Skills and Tools:
    
        a. Tools:
            - Extract specific tools mentioned in the job description, including programming languages, software applications, APIs, libraries, platforms, frameworks, cloud services, and other relevant technologies.
            - Extract both general technologies and their specific components (e.g., "AWS" and services like "EMR," "S3").            
            - Examples include Python, SQL, Postman, AWS, and CRM software.
            - Normalize tool names (e.g., "Experience with Python" becomes "Python").
            - List all identified tools, separated by commas.
    
        b. Technical Skills::
            - Extract all technical abilities, such as methodologies, domain-specific knowledge, or techniques.
            - Extract both broad technical concepts and their specific examples (e.g., "machine learning" and related algorithms like "regression," "clustering")            
            - Examples include data analysis, machine learning algorithms, financial analysis, A/B testing, data governance, clustering, regression.
            - Normalize skill names (e.g., "Proficiency in Machine Learning" should be "Machine Learning").
            - List all identified technical skills, separated by commas.

    3. Ensure Completeness and Accuracy of the extracted skills:
        - Thoroughly review the entire job description to ensure **no skills are missed**.
        - Double-check for any overlooked tools, methodologies, or competencies
        - If there is any uncertainty about a skill or its classification, include it.

    4. Format the Output in JSON:
        - Provide the parsed information in the specified JSON format, ensuring clear categorization and accurate listing.
    
    Example of the expected output:
    {
      "soft_skills": "communication, teamwork, problem-solving, leadership, adaptability",
      "tools": Python, AWS, SQL, S3, Azure
      "technical_skills": "Machine Learning, data governance , A/B testin, data visualization",
    }
    
    Job Description: {{job_description}}
    Output:
    """

In [18]:
# Build the application that will be extract soft skills and technical skills from job descriptions

fetcher = DescriptionFetcher()
prompt = PromptBuilder(template=query_template) 
llm_gpt = OpenAIGenerator(model="gpt-4o-mini")

# Create a pipeline and add the components 
gpt_extractor = Pipeline()
gpt_extractor.add_component("fetcher", fetcher)
gpt_extractor.add_component("prompt", prompt)
gpt_extractor.add_component("llm_gpt", llm_gpt)

# Create connections between components
gpt_extractor.connect("fetcher.job_info", "prompt.job_description")
gpt_extractor.connect("prompt", "llm_gpt")

🚅 Components
  - fetcher: DescriptionFetcher
  - prompt: PromptBuilder
  - llm_gpt: OpenAIGenerator
🛤️ Connections
  - fetcher.job_info -> prompt.job_description (str)
  - prompt.prompt -> llm_gpt.prompt (str)

In [19]:
# Test the application - extract skills from first rows using gpt_extractor application

gpt_replies = []

for i in range(10):
    reply = gpt_extractor.run({"fetcher": {"df": data, "row_number": i}})
    gpt_replies.append(reply["llm_gpt"]["replies"][0])

In [20]:
# Inspect reply format
gpt_replies[0]

'```json\n{\n  "soft_skills": "analytical, strategic thinking, problem-solving, communication, teamwork, adaptability, presentation, collaboration, project management",\n  "tools": "MS Project, PowerPoint, Excel, Word",\n  "technical_skills": "strategic planning, project planning, data interpretation, performance tracking, industry analysis, governance development, budget management"\n}\n```'

In [21]:
# Inspect multiple replies

for i in range(3):
    print(gpt_replies[i])

```json
{
  "soft_skills": "analytical, strategic thinking, problem-solving, communication, teamwork, adaptability, presentation, collaboration, project management",
  "tools": "MS Project, PowerPoint, Excel, Word",
  "technical_skills": "strategic planning, project planning, data interpretation, performance tracking, industry analysis, governance development, budget management"
}
```
```json
{
  "soft_skills": "communication, teamwork, problem-solving, adaptability, integrity, accountability",
  "tools": "EHR, healthcare information systems, ITIL, PMP, CHTS, CAHIMS, CPHIMS",
  "technical_skills": "data collection, audit reporting, cybersecurity, project management"
}
```
```json
{
  "soft_skills": "communication, teamwork, problem-solving, leadership, adaptability, integrity, accountability",
  "tools": "HRFISCM systems, service management systems, project management systems, content/document management systems, collaboration platforms, business enterprise solutions, IT project manage

In [22]:
# Capture text that account for the json with the target data 

pattern = r'\{.*?\}'
match = re.search(pattern, gpt_replies[0], re.DOTALL)

# Inspect match
match.group(0)

'{\n  "soft_skills": "analytical, strategic thinking, problem-solving, communication, teamwork, adaptability, presentation, collaboration, project management",\n  "tools": "MS Project, PowerPoint, Excel, Word",\n  "technical_skills": "strategic planning, project planning, data interpretation, performance tracking, industry analysis, governance development, budget management"\n}'

In [23]:
# Transform text data into dictionary

# Select text data that matches the pattern
json_string = match.group(0)

# Convert the json string to  dictionary
data_dict = json.loads(json_string)

# Inspect resulting dict
data_dict

{'soft_skills': 'analytical, strategic thinking, problem-solving, communication, teamwork, adaptability, presentation, collaboration, project management',
 'tools': 'MS Project, PowerPoint, Excel, Word',
 'technical_skills': 'strategic planning, project planning, data interpretation, performance tracking, industry analysis, governance development, budget management'}

In [ ]:
# Use this process and the dictionary keys "soft_skills", "tools', and "technical_skills" to build new columns for the dataset

# Check data lenght
len(data)

# Create batches of the data. This step is not necessary, but it might allows for more control over the process and 
# avoid timeouts or other issues that might arise when processing large amounts of data at once.
first_batch = data.iloc[0:1000]
second_batch = data.iloc[1000:2000]
third_batch = data.iloc[2000:3000]
fourth_batch = data.iloc[3000:4000]
fifth_batch = data.iloc[4000:5000]
sixth_batch = data.iloc[5000:6000]
seventh_batch = data.iloc[6000:]

In [61]:
# Use this process and the dictionary keys "soft_skills", "tools', and "technical_skills" to build new columns for the dataset

def extract_skills(batch, output_file_name, verbose=True):
    # Lists to store values
    LLM_soft_skills = []
    LLM_tools = []
    LLM_technical_skills = []

    # Repeat process shown above for all of the replies
    for i in range(len(batch)):
        reply = gpt_extractor.run({"fetcher": {"df": batch, "row_number": i}})
        # Capture text that account for the json with the target data 
        pattern = r'\{.*?\}'
        match = re.search(pattern, reply["llm_gpt"]["replies"][0], re.DOTALL)    
        # Select text data that matches the pattern if it exists
        try: 
            json_string = match.group(0)
            # Convert the json string to  dictionary
            skills_dict = json.loads(json_string)
        except:
            skills_dict = {"soft_skills": [], "tools": [], "technical_skills": []}
        # Add skills extracted to lists created previously
        try:
            LLM_soft_skills.append(skills_dict["soft_skills"])
        except:
            LLM_soft_skills.append([])
        try:
            LLM_tools.append(skills_dict["tools"])
        except:
            LLM_tools.append([])
        try:
            LLM_technical_skills.append(skills_dict["technical_skills"])
        except:
            LLM_technical_skills.append([])
        # Print progress every 100 rows if verbose is True. Just for execution control
        if (i % 100) == 0 and verbose:
            print(f"Processed {i+1} rows.")
        
    # Add skills extracted by the LLM to the current dataset
    batch["LLM_soft_skills"] = LLM_soft_skills
    batch["LLM_tools"] = LLM_tools
    batch["LLM_technical_skills"] = LLM_technical_skills

    batch.to_csv(output_file_name, index=False)

In [ ]:
# Test the function on a small batch   
test_run = first_batch.iloc[0:10]
extract_skills(test_run, "test_run.csv", verbose=True)

Processed 1 rows.


In [ ]:
# Check result
test_run[["Soft Skills", "LLM_soft_skills", "Tools", "LLM_tools", "Industry Skills", "LLM_technical_skills"]]

,Soft Skills,LLM_soft_skills,Tools,LLM_tools,Industry Skills,LLM_technical_skills
0,"['responsible', 'presentation skills', 'organi...","communication, teamwork, analytical thinking, ...",['powerpoint'],"MS Project, PowerPoint, Excel, Word",['problem-solving skills'],"strategic planning, business planning, financi..."
1,"['integrity', 'professional', 'project managem...","communication, teamwork, problem-solving, trai...",[''],"EHR, medical applications, electronic health r...",['project management'],"project management, audit reporting, data coll..."
2,"['integrity', 'professional', 'project managem...","communication, teamwork, problem-solving, lead...",[''],"HRIS, SCM, service management systems, project...",['project management'],"business enterprise solutions, cybersecurity, ..."
3,"['integrity', 'professional', 'project managem...","communication, teamwork, problem-solving, lead...","['azure', 'aws']","AWS, Azure, Cisco, TOGAF, CISSP, CCIE, PMP, ITIL",['project management'],"enterprise architecture, IT architecture, cybe..."
4,"['coaching', 'professional', 'coordination', '...","communication, teamwork, problem-solving, lead...",[''],"MS Office, recruitment software",['metrics'],"talent acquisition, recruitment analytics, job..."
5,"['integrity', 'professional', 'project managem...","communication, teamwork, problem-solving, lead...",[''],"COBIT, CGEIT, PMP, ITIL",['project management'],"IT governance, IT portfolio management, demand..."
6,"['integrity', 'professional', 'project managem...","communication, teamwork, problem-solving, lead...",['power bi'],"OMS, Power BI, data warehouse, performance man...","['business intelligence', 'project management'...","data management, data quality audits, business..."
7,"['integrity', 'responsibility', 'attention to ...","communication, teamwork, analytical thinking, ...",[''],"ITIL, CRISC, CISM","['transform', 'data governance']","risk assessment, risk management, policy devel..."
8,"['passion', 'adaptability', 'communication ski...","communication, problem-solving, teamwork, adap...",[''],ERP systems,"['problem-solving skills', 'monitoring']","supply chain management, materials planning, s..."
9,"['professional', 'resilient', 'prioritizing', ...","communication, teamwork, problem-solving, lead...",[''],CAD,[''],"drafting operations, design, technical verific..."


In [55]:
# Run on each batch of the data and save results to csv files
extract_skills(first_batch, "first_batch.csv", verbose=True)

Processed 1 rows.
Processed 101 rows.
Processed 201 rows.
Processed 301 rows.
Processed 401 rows.
Processed 501 rows.
Processed 601 rows.
Processed 701 rows.
Processed 801 rows.
Processed 901 rows.


In [71]:
extract_skills(second_batch, "second_batch.csv", verbose=True)

Processed 1 rows.
Processed 101 rows.
Processed 201 rows.
Processed 301 rows.
Processed 401 rows.
Processed 501 rows.
Processed 601 rows.
Processed 701 rows.
Processed 801 rows.
Processed 901 rows.


In [60]:
test_dict = {"soft_skills": [], "tools": [], "technical_skills": []}
test_dict["soft_skills"]

[]

In [ ]:
extract_skills(third_batch, "third_batch.csv", verbose=True)

Processed 1 rows.
Processed 101 rows.
Processed 201 rows.
Processed 301 rows.
Processed 401 rows.
Processed 501 rows.
Processed 601 rows.
Processed 701 rows.
Processed 801 rows.
Processed 901 rows.


In [63]:
data_subset3 = pd.read_csv("third_batch.csv")
data_subset3[["Soft Skills", "LLM_soft_skills", "Tools", "LLM_tools", "Industry Skills", "LLM_technical_skills"]]

,Soft Skills,LLM_soft_skills,Tools,LLM_tools,Industry Skills,LLM_technical_skills
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
995,"['teamwork', 'responsibility', 'integrity']","communication, teamwork, problem-solving, atte...",[''],"calibration equipment, testing equipment, tech...",[''],"quality control, testing methodologies, lean m..."
996,"['discipline', 'focus', 'communication', 'desi...","communication, teamwork, interpersonal skills,...",['sap'],"ProE, Creo, AutoCAD, SAP, MS Office",[''],"electrical engineering, gas insulated switchge..."
997,"['teamwork', 'responsibility', 'integrity']","communication, teamwork, problem-solving, adap...",[''],"factory equipment, calibration tools, checklis...",[''],"lean manufacturing, assembly, testing, quality..."
998,"['discipline', 'written communication', 'commu...","communication, teamwork, problem-solving, adap...","['kafka', 'azure', 'cloud']","Linux, Windows, Prometheus, Grafana, Elastic S...","['problem-solving skills', 'monitoring', 'clou...","system administration, cloud-based services, m..."


In [67]:
extract_skills(fourth_batch, "fourth_batch.csv", verbose=True)

Processed 1 rows.
Processed 101 rows.
Processed 201 rows.
Processed 301 rows.
Processed 401 rows.
Processed 501 rows.
Processed 601 rows.
Processed 701 rows.
Processed 801 rows.
Processed 901 rows.


In [68]:
extract_skills(fifth_batch, "fifth_batch.csv", verbose=True)

Processed 1 rows.
Processed 101 rows.
Processed 201 rows.
Processed 301 rows.
Processed 401 rows.
Processed 501 rows.
Processed 601 rows.
Processed 701 rows.
Processed 801 rows.
Processed 901 rows.


In [69]:
extract_skills(sixth_batch, "sixth_batch.csv", verbose=True)

Processed 1 rows.
Processed 101 rows.
Processed 201 rows.
Processed 301 rows.
Processed 401 rows.
Processed 501 rows.
Processed 601 rows.
Processed 701 rows.
Processed 801 rows.
Processed 901 rows.


In [70]:
extract_skills(seventh_batch, "seventh_batch.csv", verbose=True)

Processed 1 rows.
Processed 101 rows.
Processed 201 rows.
Processed 301 rows.


In [72]:
# Concatenate all the data subsets into a single DataFrame
data_subset1 = pd.read_csv("first_batch.csv")
data_subset2 = pd.read_csv("second_batch.csv")
data_subset3 = pd.read_csv("third_batch.csv")
data_subset4 = pd.read_csv("fourth_batch.csv")
data_subset5 = pd.read_csv("fifth_batch.csv")
data_subset6 = pd.read_csv("sixth_batch.csv")
data_subset7 = pd.read_csv("seventh_batch.csv")

# Concatenate all the data subsets into a single DataFrame
data_with_llm_skills = pd.concat([data_subset1, data_subset2, data_subset3, data_subset4, data_subset5, data_subset6, data_subset7], ignore_index=True)

In [73]:
# Transform the lists of skills into to array like format

data_with_llm_skills["LLM_soft_skills"] = data_with_llm_skills["LLM_soft_skills"].apply(lambda x: x.split(", ") if isinstance(x, str) else x)
data_with_llm_skills["LLM_tools"] = data_with_llm_skills["LLM_tools"].apply(lambda x: x.split(", ") if isinstance(x, str) else x)
data_with_llm_skills["LLM_technical_skills"] = data_with_llm_skills["LLM_technical_skills"].apply(lambda x: x.split(", ") if isinstance(x, str) else x)

In [75]:
data_with_llm_skills['key'].head()

0    15e38974831a91d5
1    9687abe64bcb08a4
2    75204b2e081fca34
3    a5b27da6109af1f5
4    8db9dcafa5ab1a00
Name: key, dtype: object

In [77]:
# Merge data contaning LLM extracted skills with full dataset

# Read full dataset (contains 'key' as job identifier. Already contains llm labels)
full_data = pd.read_csv("full_data_march_2025.csv")

full_data_with_llm_skills  = full_data.merge(data_with_llm_skills[["key", "LLM_soft_skills", "LLM_tools", "LLM_technical_skills"]], how="left", on="key")

In [80]:
# Show result in test dataset
full_data_with_llm_skills[['title', 'description', "Tools", 'Soft Skills', 'Industry Skills', 'LLM_tools', 'LLM_soft_skills', 'LLM_technical_skills']].head()

,title,description,Tools,Soft Skills,Industry Skills,LLM_tools,LLM_soft_skills,LLM_technical_skills
0,Senior Specialist - Strategy COR407,senior specialist - strategy cor407 qiddiya in...,['powerpoint'],"['responsible', 'presentation skills', 'organi...",['problem-solving skills'],"[Microsoft Project, PowerPoint, Excel, Word, s...","[analytical thinking, strategic thinking, prob...","[strategic planning, governance, project manag..."
1,SENIOR IT CLINICAL APPLICATIONS ANALYST.,job title senior it clinical applications anal...,[''],"['integrity', 'professional', 'project managem...",['project management'],"[EHR, electronic health record, ITIL, project ...","[communication, teamwork, problem-solving, lea...","[cybersecurity, audit reporting, data collecti..."
2,IT BUSINESS APPLICATIONS SPECIALIST.,job title it business applications specialist....,[''],"['integrity', 'professional', 'project managem...",['project management'],"[HRIS, SCM, project management systems, conten...","[communication, teamwork, problem-solving, men...","[business applications management, service man..."
3,SENIOR ENTERPRISE ARCHITECTURE SPECIALIST.,job title senior enterprise architecture speci...,"['azure', 'aws']","['integrity', 'professional', 'project managem...",['project management'],"[TOGAF, CISSP, AWS, Azure, CCIE, PMP, ITIL]","[communication, teamwork, problem-solving, lea...","[enterprise architecture, IT infrastructure, c..."
4,Senior Talent Acquisition Specialist,job purpose the talent acquisition specialist ...,[''],"['coaching', 'professional', 'coordination', '...",['metrics'],"[MS Office, recruitment software, talent acqui...","[communication, teamwork, problem-solving, lea...","[data analysis, recruitment analytics, talent ..."


In [ ]:
# Save data locally for reference

full_data_with_llm_skills.to_csv("Saudi_full_data_with_llm_extracted_skills.csv", index=False)